In [ ]:
!pip install requests beautifulsoup4 transformers faiss-gpu torch accelerate sentencepiece nltk playwright
from IPython.display import clear_output
clear_output()

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModel
import requests
from bs4 import BeautifulSoup
import faiss
import nltk
import numpy as np

In [ ]:
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
llm = pipeline(model="databricks/dolly-v2-3b",
               torch_dtype=torch.bfloat16,
               trust_remote_code=True,
               device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

instruct_pipeline.py:   0%|          | 0.00/9.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def scrape_page(url):
    r = requests.get(url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')

        # Extract text from <p> tags
        paragraphs = soup.find_all('p')
        all_text = ' '.join([p.text for p in paragraphs])

        # Extract images
        images = soup.find_all('img')
        image_urls = []
        for img in images:
            img_url = img.get('src')
            full_img_url = urljoin(url, img_url)
            image_urls.append(full_img_url)

            # Download the image and save it
            try:
                img_response = requests.get(full_img_url)
                img_name = os.path.join(web_dir, os.path.basename(full_img_url))
                with open(img_name, 'wb') as img_file:
                    img_file.write(img_response.content)
                print(f"Saved web scraping image: {img_name}")
            except Exception as e:
                print(f"Failed to save image from {full_img_url}: {e}")

        return all_text, image_urls
    else:
        return None, None

url = "https://github.com/CommunityOfCoders/Project-X-2023/blob/main/README.md"
content = scrape_page(url)
print("Scraped content:", content)

Scraped content: We read every piece of feedback, and take your input very seriously. 
            To see all available qualifiers, see our documentation.
           Description : This is a learning oriented project. Tranformers are the state of the art text processing models (Used as a base in all major Large Language Models (LLMs) such as GP T(ransformer) 1,2,3,4, Bard, Llama, Claude, etc). This project is an attempt to understand the transformer architecture and its working. The project is divided into 3 parts The final model we train will not be close to the current SoTA models (due to limited data and compute) but you'll gain a thorough understanding of the transformer architecture and its working so you can contribute to the real-world development and research of Transformer-based models in Open-source(which is huge! and only increasing) or otherwise. Pre-Requisites : Strong Python and C++ programming. Resources : Sequence Models-Andrew NG, Original Transformer Paper-Attenion is 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")

In [ ]:
def chunk_content(content, chunk_size=512):
    content_chunks = [content[i:i+chunk_size] for i in range(0, len(content), chunk_size)]
    return content_chunks

content_chunks = chunk_content(content)
print(f"Number of content chunks: {len(content_chunks)}")

Number of content chunks: 25


In [ ]:
def chunk_content_by_sentence(content):
    return sent_tokenize(content)

content_chunks = chunk_content_by_sentence(content)
print(f"Number of content chunks (sentences): {len(content_chunks)}")

Number of content chunks (sentences): 81


In [ ]:
chunk_embeddings = []
for chunk in content_chunks:
    inputs = tokenizer(chunk, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        embedding = model(**inputs).last_hidden_state.mean(dim=1).numpy()
    chunk_embeddings.append(embedding)

embeddings_np = np.vstack(chunk_embeddings)
print(f"Shape of embeddings: {embeddings_np.shape}")

Shape of embeddings: (81, 768)


In [ ]:
dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)
print("Number of vectors in the FAISS index:", index.ntotal)

Number of vectors in the FAISS index: 81


In [ ]:
def generate_rag_response(query, model, tokenizer, index, content_chunks):

    query_inputs = tokenizer(query, return_tensors='pt')
    with torch.no_grad():
        query_embedding = model(**query_inputs).last_hidden_state.mean(dim=1).detach().numpy()
    k = 5
    distances, indices = index.search(query_embedding, k)
    relevant_contexts = [content_chunks[i] for i in indices[0]]
    combined_context = " ".join(relevant_contexts)

    input_text = (
        f"The following context is provided to help answer the question: {combined_context}\n\n"
        f"Pay attention to the context of the question rather than just looking for similar keywords in the corpus.\n"
        f"Use three sentences maximum and keep the answer as concise as possible..\n"
        f"Question: {query}\nAnswer:"
    )
    response = llm(input_text)
    return response[0]['generated_text'].strip()

In [ ]:
query = "What is the goal of project - Three-Dimensional Reconstruction from a Single RGB Image?"
response = generate_rag_response(query, model, tokenizer, index, content_chunks)
print("Generated Response:", response)

Generated Response: The goal of this project is to develop algorithms and techniques that can take a single RGB image as input and generate a plausible 3D representation of the scene depicted in the image.
